# Calculation of diffuse fraction

For the actual albedo ($\alpha_{act} = \alpha_{ws}~f_{dif} + \alpha_{bs} (1 - f_{dif})$), we need the fraction of diffuse global radiation, calculated from the ratio of global to extraterrestrial radiation ($f_{dif}$) (Wohlfahrt et al. 2021).

To obtain $f_{dif}$, we need to:
- Get hourly extraterrestrial global radiation
- Get hourly ground-surface global radiation
- Both above were estimated wfor clear-sky atmospheric conditions (Ham in Hatfield et al. 2005, p. 549)

In [1]:
import numpy as np
import pandas as pd

In [5]:
# Main parameters, Yatir forest
latitude    = 31.346638   # latitude [degrees]
longitude   = 35.036341   # longitude [degrees]
altitude    = 653         # [m asl]
day_of_year = 180         # [1-365 or 1-366]
hour_of_day = 7           # e.g. 7.5 is 7:30 AM. Use 24h format [0.0-23.99]
timezone    = 2           # Israel: UTC+2, no DST

In [24]:
# Equation of time
# Based on Ham in Hatfield (2005)
def equation_of_time_simple(doy):
    # Time E that the sun crosses the observer's view, in min
    B = (2 * np.pi / 364) * (doy - 81)
    E = 9.87 * np.sin(2*B) - 7.53*np.cos(B) - 1.5 * np.sin(B)
    return(E)

# Equation of time
# Based on Carruthers et al. (1990) as mentioned in Ham in Hatfield (2005)
def equation_of_time_carruthers1990(doy):
    # t [radians]
    t = (2 * np.pi * doy)/366 + 4.8718
    # Time E that the sun crosses the observer's view, in min (Carruthers et al. 1990)
    E = (5.0323 - 430.847 * np.cos(t) + 12.5024 * np.cos(2*t) + 18.25 * np.cos(3*t) \
         - 100.976 * np.sin(t) + 595.275 * np.sin(2*t) + 3.6858 * np.sin(3*t) - 12.47 * np.sin(4*t))/60
    return(E)

# Hour angle [radians]
# Angular distance of the sun from the longitude of the observer
# At solar noon, the sun is directly north or south of the observer and h is zero.
def hour_angle(lon, doy, hour, tz):
    # Time E that the sun crosses the observer's view [min]
    E = equation_of_time_carruthers1990(doy)
    # Standard Meridian, 24 time zones in steps of 15°, from GMT in +/-12h
    SM = tz*15
    # Solar noon: Local standard time where solar noon occurs
    SN = 12 - E/60 - (SM - lon)/15
    # hour angle, radians
    h = (hour - SN) * np.pi / 12
    return(h)

# Sun elevation angle in radians
def sun_elevation(lat, lon, doy, hour, tz):
    # Solar declination angle (radians)
    delta = solar_declination(doy)
    # Latitude, radians
    phi = np.deg2rad(lat) # Lat in degrees
    # Hour angle [radians]
    h = hour_angle(lon, doy, hour, tz)
    
    sin_SunE = np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.cos(h)
    SunE = np.arcsin(sin_SunE)
    return(SunE)

# Solar declination angle (radians)
def solar_declination(doy):
    delta = 0.049 * np.sin(2 * np.pi / 365 * doy - 1.39)
    return(delta)

# cos_Z: Cosine of zenith angle, radians
def cos_zenith_angle(lat, lon, doy, hour, tz):
    # Solar declination angle (radians)
    delta = solar_declination(doy)
    # Latitude, radians
    phi = np.deg2rad(lat) # Lat in degrees
    # Hour angle [radians]
    h = hour_angle(lon, doy, hour, tz)
    
    cos_Z = np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.cos(h)
    return(cos_Z)

# Instantaneous (hourly) Extraterrestrial Radiation (Ham in Hatfield et al. 2005, p. 549)
def hourly_extraterrestrial_rad(lat, lon, doy, hour, tz):
    # Change naming convention
    # Day-of-year: doy is N in Hatfield (2005) [1-365 or 1-366]
    # Longitude:   lon in LOB in Hatfield (2005) [degrees]
    # Hour-of-day: hour in local time (no DST) is LST in Hatfield (2005) [0.0-23.99]
    
    # The solar constant, 1367 W m–2, 0.0820 MJ m–2 min–1
    G_sc = 1367 # W m–2
    
    # Solar declination angle (radians)
    delta = solar_declination(doy)
    # Latitude, radians
    phi = np.deg2rad(lat) # Lat in degrees
    
    # Z: Zenith angle [radians]
    cos_Z = cos_zenith_angle(lat, lon, doy, hour, tz)
    
    # The distance factor, d_r, is approximated as
    d_r = 1 + 0.033 * np.cos(2 * np.pi / 365 * doy)
    
    # Extraterrestrial radiation normal to the sun’s beam (Unused)
    R_an = G_sc * d_r
    
    # For a plane parallel to the Earth's surface (i.e., horizontal surface)
    R_a = G_sc * d_r * cos_Z
    
    return(R_a)

# Instantaneous Clear-Sky Global Irradiance: Diffuse and Direct Beam Components

# Atmospheric transmittance for beam radiation
def beam_transmittance(lat, lon, alt, doy, hour, tz):
    # Determine climate zone from doy and lat: 
    # - Mid-latitude is 23°26'22" to 66°33'39" N or S, i.e. 23.439444 to 66.560833
    #   - N summer is days 90-270, winter is 0-90 and 270-365
    #   - S summer is days 0-90 and 270-365, winter is 90-270
    # - Subarctic is
    #   - Subarctic summer is days
    # - Tropical is 23°26'22" N to 23°26'22" S
    
    # Tropical
    if(np.abs(lat) < 23.439444):
        r_0 = 0.95
        r_1 = 0.98
        r_k = 1.02
    # Midlatitude
    elif( (np.abs(lat) >= 23.439444) and (np.abs(lat) < 66.560833) ):
        # Northern Summer
        if((lat > 0) and (doy > 90) and (doy <= 270)):
            r_0 = 0.97
            r_1 = 0.99
            r_k = 1.02
        # Southern Summer
        elif((lat < 0) and ((doy <= 90) or (doy > 270))):
            r_0 = 0.97
            r_1 = 0.99
            r_k = 1.02
        # Northern winter
        elif((lat > 0) and ((doy <= 90) or (doy > 270))):
            r_0 = 0.99
            r_1 = 0.99
            r_k = 1.01
        # Southern winter
        elif((lat < 0) and (doy > 90) and (doy <= 270)):
            r_0 = 0.99
            r_1 = 0.99
            r_k = 1.01
    # Subarctic summer
    elif( ((lat >= 66.560833) and (doy > 90) and (doy <= 270)) or ((lat <= -66.560833) and ((doy <= 90) or (doy > 270)))):
        r_0 = 1.03
        r_1 = 1.01
        r_k = 1.00
    else:
        raise('No data for this climate zone!')
    
    # Parameters for beam transmittance
    a0 = r_0 * (0.4237 - 0.00821*(6.0 - alt/1000)**2)
    a1 = r_1 * (0.5055 - 0.00595*(6.5 + alt/1000)**2)
    k  = r_k * (0.2711 - 0.01858*(2.5 + alt/1000)**2)
    # cos Z: Cosine of Zenith angle [radians]
    cos_Z = cos_zenith_angle(lat, lon, doy, hour, tz)
    # Atmospheric transmittance for beam radiation
    tau_b = a0 + a1 * np.exp(-k/cos_Z)
    # Values typically range from 0.65 to 0.7 near midday.
    return(tau_b)

# Diffuse radiation, estimated from extraterrestrial radiation and its transmission coefficients
def diffuse_from_extraterrestrial(lat, lon, alt, doy, hour, tz):
    # Atmospheric transmittance for beam radiation
    tau_b = beam_transmittance(lat, lon, alt, doy, hour, tz)
    # Lui and Jordan (1960)'s rough empirical relationship between tau_b and tau_d
    tau_d = 0.2710 - 0.2939 * tau_b
    
    R_d = R_a * tau_d
    return(R_d)

# Direct beam, estimated from extraterrestrial radiation and its transmission coefficients
def beam_from_extraterrestrial(lat, lon, alt, doy, hour, tz):
    # Atmospheric transmittance for beam radiation
    tau_b = beam_transmittance(lat, lon, alt, doy, hour, tz)
    
    R_b = R_a * tau_b
    return(R_b)

def ratio_diffuse_direct_from_beams(lat, lon, alt, doy, hour, tz):
    # Diffuse radiation
    R_d = diffuse_from_extraterrestrial(lat, lon, alt, doy, hour, tz)
    # Direct beam
    R_b = beam_from_extraterrestrial(lat, lon, alt, doy, hour, tz)
    
    f_dif = R_d / R_b
    return(f_dif)

# Global irradiance
#  (i.e., horizontal surface) = sum of the beam (Rb) and diffuse components (Rd)
def global_irradiance(lat, lon, alt, doy, hour, tz):
    # Diffuse radiation
    R_d = diffuse_from_extraterrestrial(lat, lon, alt, doy, hour, tz)
    # Direct beam
    R_b = beam_from_extraterrestrial(lat, lon, doy, hour, tz)
    
    R_s = R_d + R_b
    return(R_s)


# Based on a simple empirical equation for estimating instantaneous global irradiance R_s (W m–2) from Avaste (1967)
# This equation assumes an air mass number of 1 and an atmospheric transmittance of 0.7
def global_irradiance_avaste1967(lat, lon, doy, hour, tz):
    # SunE: solar elevation angle in degrees
    SunE = sun_elevation(lat, lon, doy, hour, tz)
    SunE_deg = np.rad2deg(SunE)
    # Solar declination angle (radians)
    delta = solar_declination(doy)
    # Latitude, radians
    phi = np.deg2rad(lat) # Lat in degrees
    
    R_s = 0.8646 + 7.2396 * SunE_deg + 0.43386 * SunE_deg**2 - 6.6192 * 10**(-3) * SunE_deg**3 + 2.6677 * 10**(-5) * SunE_deg**4
    return(R_s)

# Hourly measurements of global irradiance (R_s) can be partitioned into its direct beam and diffuse components
# using several different methods (Duffie & Beckman, 1980; Weiss & Norman, 1985; Spitters et al., 1986).
# Following the approach of Spitters et al. (1986), the ratio between the diffuse component and global irradiance
# can be estimated from global irradiance (R_s), extraterrestrial radiation (R_a) and sun elevation angle (SunE)
# (Ham in Hatfield 2005, p. 552)
def ratio_diffuse_direct_simple(lat, lon, doy, hour, tz):
    # Prepare: Calculate radiation components for the estimation
    R_s = global_irradiance_avaste1967(lat, lon, doy, hour, tz)
    R_a = hourly_extraterrestrial_rad(lat, lon, doy, hour, tz)
    SunE = sun_elevation(lat, lon, doy, hour, tz)
    
    # Parameters
    R = 0.847 - 1.61 * np.sin(SunE) + 1.04 * np.sin(SunE)**2
    K = (1.47 - R)/1.66
    
    if(R_s/R_a <= 0.22):
        ratio = 1
    elif((R_s/R_a > 0.22) and (R_s/R_a <= 0.35)):
        ratio = 1.00 - 6.40 * (R_s/R_a - 0.22)**2
    elif((R_s/R_a > 0.35) and (R_s/R_a <= K)):
        ratio = 1.47 - 1.66 * R_s/R_a
    else:
        ratio = R
    return(ratio)

In [32]:
# Yatir f_dif (same method as Georg Wohlfahrt et al. 2021)
f_dif = ratio_diffuse_direct_simple(latitude, longitude, day_of_year, hour_of_day, timezone)
print('f_dif (Yatir):', f_dif)

f_dif (Yatir): 0.6889967178347249
